In [15]:
import os
import json
import pandas as pd
import traceback

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
KEY=os.getenv('OPENAI_API_KEY')

In [23]:
print(KEY)

sk-W172123456789012345678901234567890123456789012345678901234567890


In [24]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)